In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc
import pandas_profiling

In [3]:
sns.set_context('talk')

In [4]:
import warnings
warnings.filterwarnings('ignore', message='Changing the shape of non-C contiguous array')

# Read the data

In [5]:
test = pd.read_csv('csv/ech_test.csv', sep=';', index_col='id')
train = pd.read_csv('csv/ech_apprentissage.csv', sep=';', index_col='id')

/home/alexey/anaconda3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Graphs

In [6]:
train.columns

Index(['annee_naissance', 'annee_permis', 'marque', 'puis_fiscale', 'anc_veh',
       'codepostal', 'energie_veh', 'kmage_annuel', 'crm', 'profession',
       'var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9',
       'var10', 'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17',
       'var18', 'var19', 'var20', 'var21', 'var22', 'prime_tot_ttc'],
      dtype='object')

In [7]:
pandas_profiling.ProfileReport(train)

Number of variables,34
Number of observations,300000
Total Missing (%),0.0%
Total size in memory,77.8 MiB
Average record size in memory,272.0 B
Numeric,22
Categorical,9
Date,0
Text (Unique),0
Rejected,3
Distinct count,44


In [8]:
pandas_profiling.ProfileReport(test)

Number of variables,33
Number of observations,30000
Total Missing (%),0.0%
Total size in memory,7.6 MiB
Average record size in memory,264.0 B
Numeric,22
Categorical,7
Date,0
Text (Unique),0
Rejected,4
Distinct count,44


var14 оставим, но можно больше внимания уделять значению N

In [9]:
set(train['codepostal'][train['codepostal'].apply(lambda x: not str(x).isdigit())])

{'ARMEE', 'NR'}

Исследования показали, что NR есть пропуск

Выкинем лишние признаки, которые уже сильно скоррелированы 

In [10]:
drop_cols = ['annee_permis', 'var11', 'var12']

In [11]:
train = train.drop(drop_cols, axis=1)

In [12]:
test = test.drop(drop_cols, axis=1)

Заменим NR на NaN

In [13]:
train.replace('NR', np.NaN, inplace=True)
test.replace('NR', np.NaN, inplace=True)

Заменим ARMEE в codepostal на -1

In [14]:
train.loc[train.codepostal == 'ARMEE', 'codepostal'] = '-1'

In [15]:
train.loc[train.codepostal == '-1', 'codepostal']

id
46294    -1
46295    -1
46296    -1
Name: codepostal, dtype: object

In [16]:
test.codepostal.dtype

dtype('int64')

В тесте нет ARMEE и пропусков.

Переведём codepostal в числа

In [17]:
train.codepostal = pd.to_numeric(train.codepostal)

Опять посмотрим на отчёт

In [18]:
pandas_profiling.ProfileReport(train, threshold=1.1)

Number of variables,31
Number of observations,300000
Total Missing (%),0.0%
Total size in memory,71.0 MiB
Average record size in memory,248.0 B
Numeric,22
Categorical,8
Date,0
Text (Unique),0
Rejected,1
Distinct count,44


Id и codepostal сильно коррелированы, id присваивались по мере роста codepostal. Codepostal оставим, по нему будем делить на фолды

In [19]:
train.loc[train.codepostal == -1, 'codepostal']

id
46294   -1.0
46295   -1.0
46296   -1.0
Name: codepostal, dtype: float64

In [20]:
train.iloc[46296]['codepostal']

13047.0

In [21]:
train.iloc[46292]['codepostal']

13001.0

Заменим -1 на 13024, как на среднее арифметическое

In [22]:
train.loc[train.codepostal == -1, 'codepostal'] = 13024

In [23]:
train_corr = train.corr()
test_corr = test.corr()

In [24]:
sns.heatmap(train_corr)

Выкинем var1

In [25]:
train = train.drop('var1', axis=1)

In [26]:
test = test.drop('var1', axis=1)

In [27]:
train.columns

Index(['annee_naissance', 'marque', 'puis_fiscale', 'anc_veh', 'codepostal',
       'energie_veh', 'kmage_annuel', 'crm', 'profession', 'var2', 'var3',
       'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var10', 'var13',
       'var14', 'var15', 'var16', 'var17', 'var18', 'var19', 'var20', 'var21',
       'var22', 'prime_tot_ttc'],
      dtype='object')

Пропуски заменим при рассмотрении конкретных моделей. Так как пропусков не много, то не будем создавать лишние признаки (по крайней мере не здесь)

# Target label separation 

In [28]:
y_train = train['prime_tot_ttc']
train = train.drop('prime_tot_ttc', axis=1)

# Save to file

In [31]:
y_train.to_csv('preprocessed_csv/y_train.csv', sep=';')

In [32]:
train.to_csv('preprocessed_csv/train.csv', sep=';')

In [33]:
test.to_csv('preprocessed_csv/test.csv', sep=';')